In [1]:
import numpy as np
import pandas as pd
import os

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%pwd

'/home/cseadmin/dz/TrafficFlowModel/model'

In [2]:
embed_matrix=np.load("./saved/embedding_matrix.npy")

embed_matrix.shape

(492, 64)

In [3]:
cor_matrix=embed_matrix @ embed_matrix.T

cor_matrix.shape

(492, 492)

In [4]:
cor_matrix

array([[72.30105  , -0.386034 , 12.120458 , ..., -5.0983267, 10.383698 ,
         0.       ],
       [-0.386034 , 73.07714  , -8.034481 , ..., -3.3336499,  6.7834587,
         0.       ],
       [12.120458 , -8.034481 , 70.43657  , ..., -6.648742 ,  6.468872 ,
         0.       ],
       ...,
       [-5.0983267, -3.3336499, -6.648742 , ..., 76.912605 ,  2.3218575,
         0.       ],
       [10.383698 ,  6.7834587,  6.468872 , ...,  2.3218575, 77.01283  ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ]], dtype=float32)

In [15]:
# sum_of_rows=cor_matrix.sum(axis=1)
# cor_matrix/=sum_of_rows[:, np.newaxis]
# np.nan_to_num(cor_matrix, copy=False)

/home/cseadmin/dz/anaconda3/envs/torch1.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


array([[ 0.18448809,  0.00974219,  0.01123869, ...,  0.00293595,
         0.00094138,  0.        ],
       [ 0.01159629,  0.20665924,  0.04856317, ...,  0.02418181,
         0.01470489,  0.        ],
       [ 0.00733121,  0.02661366,  0.10077152, ...,  0.010441  ,
        -0.00537754,  0.        ],
       ...,
       [ 0.00251128,  0.01737695,  0.01369081, ...,  0.13979769,
         0.00495891,  0.        ],
       [ 0.00074642,  0.00979532, -0.00653646, ...,  0.00459683,
         0.12951389,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [5]:
np.save("../../DL-Traff-Graph/SZTAXI/cor_matrix.npy", cor_matrix, allow_pickle=True)

---

In [52]:
# k=4
# idx = np.argsort(cor_matrix, axis=1)[:,-k:]
# idx

array([[398, 106, 105,   0],
       [ 84, 401,  29,   1],
       [144, 127, 192,   2],
       ...,
       [208, 224, 169, 489],
       [425, 468, 243, 490],
       [156, 155, 153, 491]])

In [53]:
# knn_adj=np.zeros((492, 492))
# for i in range(len(idx)):
#     knn_adj[i][idx[i]]=1
    
# knn_adj

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [54]:
# np.save(f"../../DL-Traff-Graph/SZTAXI/adj_{k}.npy", knn_adj, allow_pickle=True)

In [ ]:
for k in range(1, 21):
    idx = np.argsort(cor_matrix, axis=1)[:,-k:]
    knn_adj=np.zeros((492, 492))
    for i in range(len(idx)):
        knn_adj[i][idx[i]]=1
    np.save(f"../../DL-Traff-Graph/SZTAXI/adj_{k}.npy", knn_adj, allow_pickle=True)

---

In [29]:
rel=pd.read_csv("../data/sz_taxi_202006/sz_taxi_202006.rel")

rel

,rel_id,type,origin_id,destination_id
0,0,geo,0,70
1,1,geo,0,246
2,2,geo,0,252
3,3,geo,1,2
4,4,geo,1,63
...,...,...,...,...
1689,1689,geo,490,244
1690,1690,geo,490,488
1691,1691,geo,491,245
1692,1692,geo,491,489


In [30]:
adj=np.zeros((492, 492))
adj_sym=np.zeros((492, 492))

for index, row in rel.iterrows():
    u=row["origin_id"]
    v=row["destination_id"]
    adj[u][v]=1
    
    adj_sym[u][v]=1
    adj_sym[v][u]=1
    
adj
adj_sym

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 0.]])

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 1., 0.]])

In [35]:
np.sum(adj[0]==1)
adj[0][105]

3

0.0

In [8]:
np.save("../data/sz_taxi_202006/adj_mx.npy", adj)
np.save("../data/sz_taxi_202006/adj_sym.npy", adj_sym)